In [ ]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas as pd 
from gerrychain.tree import recursive_tree_part
from gerrychain.updaters import Tally, cut_edges
from gerrychain.metrics import efficiency_gap, mean_median, partisan_bias, polsby_popper, partisan_gini
import geopandas
import csv, json
get_ipython().run_line_magic('matplotlib', 'inline')

In [ ]:
sen_districts = 39
sen_target_pop = sum(laprec['TotalPop'])/39
population_column = "TotalPop"
tolerance = .04

lp = Graph.from_geodataframe(laprec, ignore_errors=True)

#update with elections in your gdf
elections = [
    Election('2020_Pres',{'Democratic':'Dem_2020_Pres','Republican':'Rep_2020_Pres'}),
    Election('2019_Gov',{'Democratic':'Dem_2019_Gov','Republican':'Rep_2019_Gov'}),
    Election('2019_AG',{'Democratic':'Dem_2019_AG','Republican':'Rep_2019_AG'}),
    Election('2019_Gov.1',{'Democratic':'Dem_2019_Gov.1','Republican':'Rep_2019_Gov.1'}),
    Election('2016_Pres',{'Democratic':'Dem_2016_Pres','Republican':'Rep_2016_Pres'}),
    Election('2016_Sen',{'Democratic':'Dem_2016_Sen','Republican':'Rep_2016_Sen'}),
    Election('2016_Sen.1',{'Democratic':'Dem_2016_Sen.1','Republican':'Rep_2016_Sen.1'})
    ]

num_elections = len(elections)

election_names = [i.name for i in elections]
election_columns = [[i.parties_to_columns['Democratic'], i.parties_to_columns['Republican']] for i in elections]

In [ ]:
laprec = geopandas.read_file("G:\My Drive\SFK Maps\Final Chain Data\laprecfinal.geojson")
laprec = laprec.to_crs('epsg:3452')

#laprec.to_file("G:\\My Drive\\SFK Maps\\Oct 21\\laprecfinal.geojson", driver='GeoJSON')  
#laprec.to_file("G:\\My Drive\\SFK Maps\\Oct 21\\laprecfinal.geojson", driver='GeoJSON')
#laprec.to_csv("G:\My Drive\SFK Maps\Check of percmetrics\laprec.csv")

# Population updater, for computing how close to equality the district
# populations are. "TOTPOP" is the population column from our shapefile.
my_updaters = {"population": updaters.Tally("TotalPop", alias="population") }

In [ ]:
def num_splits(partition): 
    laprec["current"] = laprec.index.map(dict(partition.assignment))
    splits = sum(laprec.groupby("COUNTYFP10")["current"].nunique() > 1)
    return splits

In [ ]:
# In[2]:


# Election updaters, for computing election results using the vote totals
# from our shapefile.
election_updaters = {election.name: election for election in elections}

my_updaters.update(election_updaters)

sen_partition = GeographicPartition(lp, assignment="sen_seed", updaters=my_updaters)

sencompactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(sen_partition["cut_edges"])
)

senpop_constraint = constraints.within_percent_of_ideal_population(sen_partition, 0.05)


senproposal = partial(recom,
                   pop_col="TotalPop",
                   pop_target=sen_target_pop,
                   epsilon=0.04,
                   node_repeats=2
                  )
senchain = MarkovChain(
    proposal=senproposal,
    constraints=[
        senpop_constraint,
        sencompactness_bound
    ],
    accept=accept.always_accept,
    initial_state=sen_partition,
    total_steps=50000
)

unique_label = "GEOID20"
pop_col = "TotalPop"
district_col = "sen_seed"
county_col = "COUNTYFP10"


    


# In[3]:



In [ ]:
def percmetrics(partnum, partition):
    global laprec
    laprec["current"] = laprec.index.map(dict(partition.assignment))
    distpop = laprec.groupby(["current"])["Total_2020_VAP"].sum()
    dempop = laprec.groupby(["current"])["Total_2016-2020_Comp"].sum()
    dpdf = pandas.concat([distpop,dempop], axis=1)
    dpdf = dpdf.rename(columns={"Total_2020_VAP":"temppop", "Total_2016-2020_Comp":"dempop"})
    laprec = laprec.merge(dpdf, how = "left", left_on="current", right_on = dpdf.index)
    laprec["wgtav"] = laprec["Total_2020_VAP"] / laprec["temppop"]
    laprec['dwgt'] = laprec["Total_2016-2020_Comp"] / laprec["dempop"]
    laprec["BlkPct"] = laprec["Black_2020_VAP"] /  laprec["Total_2020_VAP"] 
    laprec["MinPct"] = (laprec["Total_2020_VAP"] - laprec["White_2020_VAP"] ) /  laprec["Total_2020_VAP"] 
    laprec["DemPct"] = laprec["Dem_2016-2020_Comp"] / laprec["Total_2016-2020_Comp"] 
    laprec["BlkPctW"] = laprec["BlkPct"] * laprec["wgtav"]
    laprec["MinPctW"] = laprec["MinPct"] * laprec["wgtav"]
    laprec["DemPctW"] = laprec["DemPct"] * laprec["dwgt"]
    DistDemPct = laprec.groupby(["current"])["DemPctW"].sum()
    DistMinPct = laprec.groupby(["current"])["MinPctW"].sum()
    DistBlkPct = laprec.groupby(["current"])["BlkPctW"].sum()
    DistDemPct = pandas.concat([DistDemPct, DistMinPct, DistBlkPct], axis=1)
    DistDemPctD = DistDemPct.to_dict()
    findict = {partnum: DistDemPctD}
    del(laprec["temppop"])
    del(laprec["dempop"])
    return findict

In [ ]:
# In[ ]:


pop_vec = []
cut_vec = []
votes = [[], [], [], [], [], [], [], [], [], [], [], [], [], []]
mms = []
egs = []
hmss = []
pgs =[]
pbs = []
splits = []
pcents = []

f=  open("G:\\My Drive\\SFK Maps\\Final Chain Data\\Senate\\Partitions\\tracker.txt", "a+")

for partnum, part in enumerate(senchain):
    
    percd =  percmetrics(partnum,part)
    
    dct = 0
    mmct = 0
    dem = 0
    
    
    for key in percd[partnum]['DemPctW']:
        if percd[partnum]['DemPctW'][key] >= 0.5:
            dem += 1
        else:
            pass
        
    for key in percd[partnum]['DemPctW']:
        if 0.45 <= percd[partnum]['DemPctW'][key] <= 0.55:
            dct += 1
        else:
            pass
        
    for key in percd[partnum]['MinPctW']:
        if percd[partnum]['MinPctW'][key] >= 0.50:
            mmct += 1
        else:
            pass
    
    if dem >= 9 and mmct >= 9 and dct >= 5 :
        pcents.append(percd)
        sens = []
        g = list(part.graph.nodes)
        for y in g:
            tt = []
            h = part.assignment[y]
            tt.append(y)
            tt.append(h)
            sens.append(tt)
        
        sensdf = pandas.DataFrame(sens)
        sensdf.to_csv("G:\\My Drive\\SFK Maps\\Final Chain Data\\Senate\\Partitions\\" +"partassgn"+ str(partnum)+ ".csv") 
    
        splits.append(num_splits(part))
    
        pop_vec.append(sorted(list(part["population"].values())))
        cut_vec.append(len(part["cut_edges"]))
        mms.append([])
        egs.append([])
        hmss.append([])
        pgs.append([])
        pbs.append([])
        
        for elect in range(num_elections):
            votes[elect].append(sorted(part[election_names[elect]].percents("Democratic")))
            mms[-1].append(mean_median(part[election_names[elect]]))
            egs[-1].append(efficiency_gap(part[election_names[elect]]))
            pgs[-1].append(partisan_gini(part[election_names[elect]]))
            pbs[-1].append(partisan_bias(part[election_names[elect]]))
            hmss[-1].append(part[election_names[elect]].wins("Democratic"))
        print(partnum, mmct, dem, dct,"!!!!!")
        f.write(str(partnum) +" "+ str(mmct) +" "+ str(dem) +" "+ str(dct)+" "+ "\n")
    else:
        print(partnum, mmct,dem, dct)
        
f.close()

# In[6]:

In [ ]:
mmspd = pd.DataFrame(mms)
egspd = pd.DataFrame(egs)
pgspd = pd.DataFrame(pgs)
pbspd = pd.DataFrame(pbs)
hmsspd = pd.DataFrame(hmss)
splpd = pd.DataFrame(splits)
compctpd = pd.DataFrame(compct)

mmspd = mmspd.rename(columns={0: "20Pres_mms",1: "19Gov_mms",2: "19AG_mms",3: "19Gov_1_mms",4: "16Pres_mms",5: "16sen_mms",6: "16_sen_1_mms"})
pgspd = pgspd.rename(columns={0: "20Pres_pgs",1: "19Gov_pgs",2: "19AG_pgs",3: "19Gov_1_pgs",4: "16Pres_pgs",5: "16sen_pgs",6: "16_sen_1_pgs"})
pbspd = pbspd.rename(columns={0: "20Pres_pbs",1: "19Gov_pbs",2: "19AG_pbs",3: "19Gov_1_pbs",4: "16Pres_pbs",5: "16sen_pbs",6: "16_sen_1_pbs"})
egspd = mmspd.rename(columns={0: "20Pres_egs",1: "19Gov_egs",2: "19AG_egs",3: "19Gov_1_egs",4: "16Pres_egs",5: "16sen_egs",6: "16_sen_1_egs"})
hmsspd = hmsspd.rename(columns={0: "20Pres_hmss",1: "19Gov_hmss",2: "19AG_hmss",3: "19Gov_1_hmss",4: "16Pres_hmss",5: "16sen_hmss",6: "16_sen_1_hmss"})
splpd = splpd.rename(columns={0: "ParishSplits"})

findf = pandas.concat([compctpd, mmspd,hmsspd,pgspd,pbspd,splpd],axis = 1)


findf = findf.rename(columns={0: "ChainNum", 1: "CompDist",2: "DemMaj",3: "NonPckedBlck",4: "BlckMaj",5: "NonPckedMM",6: "MM"})

#findf.to_csv("G:\My Drive\SFK Maps\Final Chain Data\lasen.csv")


# In[50]:


#laprec.to_csv("G:\My Drive\SFK Maps\Final Chain Data\laprec.csv")


# In[8]:


findf.to_csv("G:\\My Drive\\SFK Maps\\Final Chain Data\\Senate\\lasen.csv")


# In[9]:


